In [116]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Data Collection

### Files sources

In [159]:
# URLs of the files
train_datas_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/module4_exercise_train.zip'
test_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/Neighborhood_Market_data.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_datas_url, 'module4_exercise_train.zip')
download_file(test_data_url, 'Neighborhood_Market_data.csv')

Downloaded module4_exercise_train.zip from https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/module4_exercise_train.zip
Downloaded Neighborhood_Market_data.csv from https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/Neighborhood_Market_data.csv


In [160]:

import zipfile

with zipfile.ZipFile('module4_exercise_train.zip', 'r') as zip_ref:
    zip_ref.extractall('data_train')

#### CityMart

In [161]:
# read "CityMart_data.csv"
df_CityMart = pd.read_csv("data_train/CityMart_data.csv", sep=",", index_col='item_code')
df_CityMart

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified
item_code,,,,,,,,,,
P0019,CityMart,2.81,26.83,38.75,24.89,323,25877.199625,253,202,2023-01-19
P0024,CityMart,3.30,59.23,34.99,21.78,321,45138.128706,21,225,2023-01-24
P0025,CityMart,2.34,22.60,16.90,60.12,291,22962.232800,316,278,2023-01-25
P0034,CityMart,6.54,18.59,68.72,21.99,126,28092.330552,612,233,2023-02-03
P0039,CityMart,9.94,57.89,88.33,35.45,312,181270.870165,968,203,2023-02-08
...,...,...,...,...,...,...,...,...,...,...
P1984,CityMart,8.24,99.78,68.06,11.87,117,80609.488116,333,263,2028-06-06
P1989,CityMart,5.79,71.19,92.39,63.98,250,420812.077518,417,238,2028-06-11
P1992,CityMart,8.64,97.57,8.58,83.35,348,69776.502510,489,219,2028-06-14


#### Greenfield_Grocers

In [162]:
# read "Greenfield_Grocers_data.csv"
df_Greenfield_Grocers = pd.read_csv("data_train/Greenfield_Grocers_data.csv", sep="|", index_col='ITEM_CODE', skiprows=3, usecols=range(0, 12))
df_Greenfield_Grocers.columns = df_Greenfield_Grocers.columns.str.lower()
# Drop the last column
df_Greenfield_Grocers = df_Greenfield_Grocers.iloc[:, :-1]
df_Greenfield_Grocers

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified
ITEM_CODE,,,,,,,,,,
P0006,Greenfield_Grocers,5.02,86.68,71.64,16.42,66,101964.180384,450,130,2023-01-06
P0014,Greenfield_Grocers,9.91,21.67,54.91,84.80,99,100903.494560,827,118,2023-01-14
P0016,Greenfield_Grocers,1.13,60.03,97.39,90.00,348,526168.953000,981,136,2023-01-16
P0021,Greenfield_Grocers,0.95,40.36,67.91,99.76,301,273426.956576,289,155,2023-01-21
P0028,Greenfield_Grocers,5.24,22.37,61.78,68.67,15,94903.217262,423,177,2023-01-28
...,...,...,...,...,...,...,...,...,...,...
P1983,Greenfield_Grocers,3.13,37.39,74.87,84.11,168,235456.634023,31,154,2028-06-05
P1985,Greenfield_Grocers,0.66,30.04,15.77,70.62,5,33454.869096,76,73,2028-06-07
P1988,Greenfield_Grocers,7.28,7.34,89.94,37.33,92,24643.757868,770,162,2028-06-10


#### Outlet_data

In [163]:
# read "SuperSaver_Outlet_data.xlsx"

df_SuperSaver_Outlet = pd.read_excel("data_train/SuperSaver_Outlet_data.xlsx", index_col='item_code')
df_SuperSaver_Outlet

,quantity_sold
item_code,
P0003,198
P0007,211
P0008,200
P0009,209
P0012,186
...,...
P1976,236
P1980,192
P1986,193


#### HighStreet_Bazaar

In [164]:

df_HighStreet_Bazaar = pd.read_json("data_train/HighStreet_Bazaar_data.json", orient='records')
df_HighStreet_Bazaar.set_index('item_code', inplace=True)
df_HighStreet_Bazaar['last_modified'] = pd.to_datetime(df_HighStreet_Bazaar['last_modified'], unit='ms')
df_HighStreet_Bazaar

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified
item_code,,,,,,,,,,
P0001,HighStreet_Bazaar,6.11,75.46,91.62,92.08,78.0,636608.450016,237,346,2023-01-01
P0011,HighStreet_Bazaar,4.34,16.97,93.21,54.58,344.0,86333.208546,184,218,2023-01-11
P0015,HighStreet_Bazaar,1.37,58.93,11.28,73.87,344.0,49103.634648,946,315,2023-01-15
P0017,HighStreet_Bazaar,7.27,51.51,39.21,16.17,138.0,32658.663807,268,228,2023-01-17
P0020,HighStreet_Bazaar,0.89,57.50,69.84,6.12,333.0,24576.696000,396,228,2023-01-20
...,...,...,...,...,...,...,...,...,...,...
P1978,HighStreet_Bazaar,6.28,72.48,47.71,83.54,322.0,288883.057632,637,235,2028-05-31
P1981,HighStreet_Bazaar,0.91,31.13,55.22,71.22,342.0,122427.080292,60,241,2028-06-03
P1982,HighStreet_Bazaar,4.65,65.72,82.01,60.75,192.0,327424.104900,925,237,2028-06-04


In [166]:
data = pd.concat([df_CityMart, df_Greenfield_Grocers, df_SuperSaver_Outlet, df_HighStreet_Bazaar], axis=0)


data

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified
P0019,CityMart,2.81,26.83,38.75,24.89,323.0,25877.199625,253.0,202,2023-01-19
P0024,CityMart,3.30,59.23,34.99,21.78,321.0,45138.128706,21.0,225,2023-01-24
P0025,CityMart,2.34,22.60,16.90,60.12,291.0,22962.232800,316.0,278,2023-01-25
P0034,CityMart,6.54,18.59,68.72,21.99,126.0,28092.330552,612.0,233,2023-02-03
P0039,CityMart,9.94,57.89,88.33,35.45,312.0,181270.870165,968.0,203,2023-02-08
...,...,...,...,...,...,...,...,...,...,...
P1978,HighStreet_Bazaar,6.28,72.48,47.71,83.54,322.0,288883.057632,637.0,235,2028-05-31 00:00:00
P1981,HighStreet_Bazaar,0.91,31.13,55.22,71.22,342.0,122427.080292,60.0,241,2028-06-03 00:00:00
P1982,HighStreet_Bazaar,4.65,65.72,82.01,60.75,192.0,327424.104900,925.0,237,2028-06-04 00:00:00
P1987,HighStreet_Bazaar,4.64,93.41,52.53,41.02,131.0,201278.055846,317.0,256,2028-06-09 00:00:00


#### Aggregate

#### Simple baseline

In [167]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def get_simple_baseline(data, fillna_value=-1, drop_cols=None, k_fold=5, scaler='standard', model='linear', metric='mae', target_col=None, X_data_test=None):

    data = data.copy()
    # Handle missing values
    data.fillna(fillna_value, inplace=True)
    if X_data_test is not None:
        X_data_test = X_data_test.copy()
        X_data_test.fillna(fillna_value, inplace=True)

    # Drop unwanted columns
    if drop_cols:
        data.drop(drop_cols, axis=1, inplace=True)
        if X_data_test is not None:
            X_data_test.drop(drop_cols, axis=1, inplace=True)

    # Split data into features (X) and target (y)
    y = data[target_col]
    X = data.drop(target_col, axis=1)

    # Feature scaling
    if scaler == 'standard':
        scaler = StandardScaler()
    elif scaler == 'minmax':
        scaler = MinMaxScaler()
    else:
        scaler = None

    if scaler:
        X = scaler.fit_transform(X)
        if X_data_test is not None:
            X_data_test = scaler.transform(X_data_test)

    # Initialize the model
    if model == 'linear':
        model = LinearRegression()
    elif model == 'logistic':
        model = LogisticRegression()
    elif model == 'random_forest':
        model = RandomForestClassifier()
    else:
        raise ValueError("Unsupported model type")

    # Initialize cross-validation
    kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)
    scores = []

    # Train and evaluate using k-fold cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Evaluate using the specified metric
        if metric == 'mae':
            score = mean_absolute_error(y_test, y_pred)
        elif metric == 'accuracy':
            score = accuracy_score(y_test, np.round(y_pred))
        else:
            raise ValueError("Unsupported metric")

        scores.append(score)

    if X_data_test is not None:
        model.fit(X, y)
        return np.mean(scores), model.predict(X_data_test)

    # Return the average score
    return np.mean(scores)

In [169]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

np.float64(45.252609628391504)

### API sources

In [170]:
def get_api(endpoint_url):
    try:
        # Make the GET request to the mock API
        response = requests.get(endpoint_url)

        if response.status_code == 200:
            data = response.json()
            print(data["message"])
            return data['data']
        else:
            print(f"Failed to retrieve volume data. Status code: {response.status_code}")

    except Exception as e:
        print(f"An error occurred: {e}")

password = get_api("https://www.raphaelcousin.com/api/exercise/auth")["password"]
prices = get_api(f"https://www.raphaelcousin.com/api/exercise/{password}/prices")

Authentication successful
Volume data retrieved successfully


In [171]:
df_prices = pd.DataFrame.from_dict(prices, orient='index', columns=['prices'])
df_prices.head()

,prices
P0001,22.14
P0002,26.91
P0003,16.90
P0004,7.04
P0005,20.84


#### Aggregate

In [172]:
data = pd.merge(data, df_prices, left_index=True, right_index=True, how='left')
data

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified,prices
P0019,CityMart,2.81,26.83,38.75,24.89,323.0,25877.199625,253.0,202,2023-01-19,17.86
P0024,CityMart,3.30,59.23,34.99,21.78,321.0,45138.128706,21.0,225,2023-01-24,14.04
P0025,CityMart,2.34,22.60,16.90,60.12,291.0,22962.232800,316.0,278,2023-01-25,3.38
P0034,CityMart,6.54,18.59,68.72,21.99,126.0,28092.330552,612.0,233,2023-02-03,15.11
P0039,CityMart,9.94,57.89,88.33,35.45,312.0,181270.870165,968.0,203,2023-02-08,14.20
...,...,...,...,...,...,...,...,...,...,...,...
P1978,HighStreet_Bazaar,6.28,72.48,47.71,83.54,322.0,288883.057632,637.0,235,2028-05-31 00:00:00,15.85
P1981,HighStreet_Bazaar,0.91,31.13,55.22,71.22,342.0,122427.080292,60.0,241,2028-06-03 00:00:00,17.98
P1982,HighStreet_Bazaar,4.65,65.72,82.01,60.75,192.0,327424.104900,925.0,237,2028-06-04 00:00:00,17.75
P1987,HighStreet_Bazaar,4.64,93.41,52.53,41.02,131.0,201278.055846,317.0,256,2028-06-09 00:00:00,22.68


In [173]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

np.float64(44.44496732780653)

### Scrapping sources

In [174]:
# Full setup in one cell
!pip install selenium

# Install Chrome
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://cli.github.com/packages stable InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [175]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def create_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")

    return webdriver.Chrome(options=chrome_options)

# Usage example
driver = create_driver()

In [178]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time


from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def create_driver() :
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')


    return webdriver.Chrome(options=chrome_options)

# Set up the Selenium WebDriver (e.g., Chrome)
driver = create_driver()
# driver = webdriver.Chrome()  # Make sure ChromeDriver is installed
# driver = webdriver.Firefox()
# driver = webdriver.Edge()
# driver = webdriver.Safari()

# Open the URL
url = 'https://www.raphaelcousin.com/module4/scrapable-data'
driver.get(url)

# Wait for the page to fully load (increase time if needed)
time.sleep(5)

# Get the fully rendered page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Initialize lists to store scraped data
exercise_data = []

# Find both tables
tables = soup.find_all('table')

# Close the Selenium WebDriver
driver.quit()

# Scrape the second table (Exercise Data)
course_table = tables[1]
for row in course_table.find('tbody').find_all('tr'):
    cols = row.find_all('td')
    #exercise_data.append({ TODO })

# Convert the lists to pandas DataFrames
df_exercise = pd.DataFrame(exercise_data)
df_exercise


""


#### Aggregate

In [179]:
data = pd.merge(data, df_exercise, left_index=True, right_index=True, how='left')

In [180]:
data

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,quantity_sold,last_modified,prices
P0019,CityMart,2.81,26.83,38.75,24.89,323.0,25877.199625,253.0,202,2023-01-19,17.86
P0024,CityMart,3.30,59.23,34.99,21.78,321.0,45138.128706,21.0,225,2023-01-24,14.04
P0025,CityMart,2.34,22.60,16.90,60.12,291.0,22962.232800,316.0,278,2023-01-25,3.38
P0034,CityMart,6.54,18.59,68.72,21.99,126.0,28092.330552,612.0,233,2023-02-03,15.11
P0039,CityMart,9.94,57.89,88.33,35.45,312.0,181270.870165,968.0,203,2023-02-08,14.20
...,...,...,...,...,...,...,...,...,...,...,...
P1978,HighStreet_Bazaar,6.28,72.48,47.71,83.54,322.0,288883.057632,637.0,235,2028-05-31 00:00:00,15.85
P1981,HighStreet_Bazaar,0.91,31.13,55.22,71.22,342.0,122427.080292,60.0,241,2028-06-03 00:00:00,17.98
P1982,HighStreet_Bazaar,4.65,65.72,82.01,60.75,192.0,327424.104900,925.0,237,2028-06-04 00:00:00,17.75
P1987,HighStreet_Bazaar,4.64,93.41,52.53,41.02,131.0,201278.055846,317.0,256,2028-06-09 00:00:00,22.68


In [181]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

np.float64(44.44496732780653)

### Generating Submission File

In [183]:
# X_test =  read  Neighborhood_Market_data

# read
df_StoreN =  pd.read_csv("Neighborhood_Market_data.csv", sep=",", index_col='item_code')
df_StoreN

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,last_modified
item_code,,,,,,,,,
P0002,Neighborhood_Market,5.51,51.79,46.72,72.02,344,174261.666176,287,2023-01-02
P0004,Neighborhood_Market,3.97,84.63,39.42,42.46,189,141651.425916,387,2023-01-04
P0005,Neighborhood_Market,5.99,39.33,83.51,5.12,183,16816.375296,382,2023-01-05
P0010,Neighborhood_Market,4.10,77.43,49.56,74.41,208,285543.225828,656,2023-01-10
P0013,Neighborhood_Market,6.96,95.39,34.61,23.24,114,76725.649196,755,2023-01-13
...,...,...,...,...,...,...,...,...,...
P1972,Neighborhood_Market,5.68,17.89,71.87,95.67,296,123008.113881,411,2028-05-25
P1977,Neighborhood_Market,1.53,84.32,64.91,66.30,171,362873.902560,702,2028-05-30
P1991,Neighborhood_Market,7.79,84.46,82.96,52.14,50,365334.635424,190,2028-06-13


In [184]:
df_StoreN = pd.merge(df_StoreN, df_prices, left_index=True, right_index=True, how='left')
df_StoreN = pd.merge(df_StoreN, df_exercise, left_index=True, right_index=True, how='left')

In [185]:
 _, x_pred = get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold', X_data_test = df_StoreN)

In [186]:
submission = pd.DataFrame({
    'item_code': df_StoreN.index,
    'quantity_sold': x_pred # your_prediction
})

submission.to_csv('submission.csv', index=False, sep=',')
submission.head()

,item_code,quantity_sold
0,P0002,178.555256
1,P0004,219.019966
2,P0005,185.850608
3,P0010,222.778341
4,P0013,209.470535


In [187]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>